In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings

In [4]:
!export OLLAMA_HOST=localhost:8888
!ollama serve
!ollama pull mxbai-embed-large

Error: listen tcp 127.0.0.1:11434: bind: address already in use
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠸ pulling manifest ⠸ pulling manifest ⠴ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest 
pulling 819c2adf5ce6...   0% ▕                ▏    0 B/669 MB                  pulling manifest 
pulling 819c2adf5ce6...   0% ▕                ▏    0 B/669 MB                  pulling manifest 
pulling 819c2adf5ce6...   0% ▕                ▏    0 B/669 MB                  pulling manifest 
pulling 819c2adf5ce6...   0% ▕                ▏    0 B/669 MB                  pulling manifest 
pulling 819c2adf5ce6...   0% ▕                ▏    0 B/669 MB                  pulling manifest 
pulling 819c2adf5ce6...   0% ▕                ▏    0 B/669 MB                  pulling manifest 
pulling 819c2adf5ce6...   0% ▕                ▏    0 B/669 MB                  pulling manifest 
pulling 819

In [6]:
if __name__=="__main__":
        DB_FAISS_PATH = '../vectorstore'
        loader=PyPDFLoader("random machine learing pdf.pdf")
        docs=loader.load()
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        splits = text_splitter.split_documents(docs)
        vectorstore = FAISS.from_documents(documents=splits, embedding=OllamaEmbeddings(model="mxbai-embed-large", show_progress=True))
        vectorstore.save_local(DB_FAISS_PATH)

OllamaEmbeddings: 100%|██████████| 338/338 [05:57<00:00,  1.06s/it]
